In [1]:
# Import sqllite libraries
import sqlite3 
import requests

In [2]:
#Import Yahoo Finance + Pandas and other libraries 

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
#data=yf.download('AMZN',start='2020-06-01', interval='1h',  end='2021-06-01',progress=False)[['Close']]

tickerStrings = ['AAPL', 'MSFT', 'GOOG', 'GE', 'BA', 'DAL', 'IBM']
df = yf.download(tickerStrings, group_by='Ticker', period='1000d')
df = df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)


[*********************100%***********************]  7 of 7 completed


In [27]:
df.head(1000)


,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2017-07-19,AAPL,35.945583,37.755001,37.855000,37.487499,37.619999,83692000
2017-07-19,BA,198.707397,210.880005,210.929993,208.990005,209.429993,2258400
2017-07-19,DAL,50.100189,53.790001,54.340000,53.099998,53.770000,10733000
2017-07-19,GE,24.509535,25.903847,25.990385,25.807692,25.817308,25784096
2017-07-19,GOOG,970.890015,970.890015,973.039978,964.030029,967.840027,1224500
...,...,...,...,...,...,...,...
2018-02-09,BA,318.750305,332.829987,337.809998,319.250000,333.549988,8833000
2018-02-09,DAL,47.596436,50.459999,52.000000,48.520000,51.950001,19053400
2018-02-09,GE,13.822739,14.365385,14.451923,13.682692,14.125000,134218032


In [28]:
df.reset_index(inplace=True)

In [29]:
df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2017-07-19,AAPL,35.945583,37.755001,37.855000,37.487499,37.619999,83692000
1,2017-07-19,BA,198.707397,210.880005,210.929993,208.990005,209.429993,2258400
2,2017-07-19,DAL,50.100189,53.790001,54.340000,53.099998,53.770000,10733000
3,2017-07-19,GE,24.509535,25.903847,25.990385,25.807692,25.817308,25784096
4,2017-07-19,GOOG,970.890015,970.890015,973.039978,964.030029,967.840027,1224500


In [30]:
#Rename Columns in DataFrame

df.rename(columns = {'Adj Close':'AdjClose'}, inplace = True)
df.rename(columns = {'Date':'TrxDate'}, inplace = True)

In [31]:
df.head()

,TrxDate,Ticker,AdjClose,Close,High,Low,Open,Volume
0,2017-07-19,AAPL,35.945583,37.755001,37.855000,37.487499,37.619999,83692000
1,2017-07-19,BA,198.707397,210.880005,210.929993,208.990005,209.429993,2258400
2,2017-07-19,DAL,50.100189,53.790001,54.340000,53.099998,53.770000,10733000
3,2017-07-19,GE,24.509535,25.903847,25.990385,25.807692,25.817308,25784096
4,2017-07-19,GOOG,970.890015,970.890015,973.039978,964.030029,967.840027,1224500


In [32]:
# Establish sqllite3 connectivity

conn = sqlite3.connect('stocktrx.db')
c = conn.cursor()

In [33]:
#Drop Tables
c.execute ("DROP Table stocks")

In [34]:
#create coins table
c.execute("CREATE TABLE stocks (TrxDate text, Ticker text, Open REAL, High REAL, Low REAL, Close REAL, AdjClose REAL,Volume REAL )")

In [35]:
# Insert into Coins Table
#c.execute("insert into stocks values (?,?,?,?,?,?,?)", ['2017-01-03', 28.950001,29.082500,28.690001,29.03750, 27.413372,115127600 ] )

In [36]:
# Pull data from Yahoo Finance DataFrame
table_name='stocks'
df.to_sql(table_name, conn, if_exists='append', index=False)

In [37]:
#Check if data exiss or not in the stocks table

for row in c.execute('SELECT distinct ticker FROM stocks;'):
    print(row)

('AAPL',)
('BA',)
('DAL',)
('GE',)
('GOOG',)
('IBM',)
('MSFT',)


In [38]:
#commit and Close session

conn.commit()
conn.close()